In [ ]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from keras import layers
from keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
path= '/kaggle/input/utkface-new/UTKFace'
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

In [ ]:
os.chdir(path)
file_list = os.listdir() #Go path and list files
print("Number Of İmages: ", len(file_list))
#Gender
gender = []
for i in file_list:
    gender.append(i.split('_')[1])

gender_labels= ["Male","Female"]

gender_classes = []
y_gender= []
for i in gender:
    i= int(i)
    if i== 0:
        gender_classes.append(0)
    else:
        gender_classes.append(1)
    y_gender.append(i)
    
gender_classes= np.array(gender_classes)
print("gender_classes shape: ",gender_classes.shape) 

In [ ]:
## Age binning
age = []
for i in file_list:
    if int(i.split("_")[0])<=5:
        age.append(0)
    elif int(i.split("_")[0])> 5 & int(i.split("_")[0]) <=10:
        age.append(1)
    elif int(i.split("_")[0])> 10 & int(i.split("_")[0]) <=15:
        age.append(2)
    elif int(i.split("_")[0])> 15 & int(i.split("_")[0]) <=20:
        age.append(3)
    elif int(i.split("_")[0])> 20 & int(i.split("_")[0]) <=25:
        age.append(4)
    elif int(i.split("_")[0])> 25 & int(i.split("_")[0]) <=30:
        age.append(5)
    elif int(i.split("_")[0])> 30 & int(i.split("_")[0]) <=35:
        age.append(6)
    elif int(i.split("_")[0])> 35 & int(i.split("_")[0]) <=40:
        age.append(7)
    elif int(i.split("_")[0])> 40 & int(i.split("_")[0]) <=45:
        age.append(8)
    elif int(i.split("_")[0])> 45 & int(i.split("_")[0]) <=50:
        age.append(9)
    elif int(i.split("_")[0])> 50 & int(i.split("_")[0]) <=55:
        age.append(10)
    elif int(i.split("_")[0])> 55 & int(i.split("_")[0]) <=60:
        age.append(11)
    elif int(i.split("_")[0])> 60 & int(i.split("_")[0]) <=65:
        age.append(12)
    elif int(i.split("_")[0])> 65 & int(i.split("_")[0]) <=70:
        age.append(13)
    elif int(i.split("_")[0])> 70 & int(i.split("_")[0]) <=75:
        age.append(14)
    elif int(i.split("_")[0])> 75 & int(i.split("_")[0]) <=80:
        age.append(15)
    elif int(i.split("_")[0])> 80:
        age.append(16)

age = np.array(age)
print(age.shape)

In [ ]:
x_data=[]
for file in file_list:
    img= cv2.imread(file)
    img=cv2.resize(img,dsize = (128,128)) # (200,200)--->(48,48)
    x_data.append(img)
x_data= np.array(x_data)
print("x_data shape: ",x_data.shape)

In [ ]:
## Train-Test Split for gender
x_train,x_test,y_train,y_test=train_test_split(x_data,gender_classes,test_size=0.2,
                                               shuffle=True,random_state=42)

#one hot encoding
y_train= to_categorical(y_train,num_classes=2)
y_test= to_categorical(y_test,num_classes=2) 

print("Samples in Training:",x_train.shape)
print("Samples in Testing:",x_test.shape)


print("y_train shape: ",y_train.shape)
print("y_test shape: ",y_test.shape)

input_shape= x_train.shape[1:]
print("input shape: ",input_shape)

In [ ]:
from keras.applications.vgg16 import VGG16 

vgg= VGG16(include_top= False,weights="imagenet",input_shape=input_shape) ## using pre-trained ImageNet weights

print(vgg.summary())

In [ ]:
vgg_layer_list= vgg.layers 

model_gender= Sequential()

for layer in vgg_layer_list: 
    model_gender.add(layer)     

for layer in model_gender.layers:
    layer.trainable= False 

model_gender.add(layers.Flatten())
model_gender.add(layers.Dense(512, activation="relu"))
model_gender.add(layers.Dropout(0.5))
model_gender.add(layers.Dense(256, activation="relu"))
model_gender.add(layers.Dense(128, activation="relu"))
model_gender.add(layers.Dense(64, activation="relu"))
model_gender.add(layers.Dense(32, activation="relu"))
model_gender.add(layers.Dense(2, activation="sigmoid"))

print(model_gender.summary())

In [ ]:
# run model for gender
model_gender.compile(loss="binary_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

gender = model_gender.fit(x_train,y_train,
                validation_data=(x_test,y_test),
                epochs=10,
                batch_size= 8)

In [ ]:
#Train and Validation Loss
plt.plot(gender.history["loss"],label="train loss")
plt.plot(gender.history["val_loss"],label="val loss")
plt.legend()
plt.show()
plt.draw()

##Train and Validation Accuracy
plt.plot(gender.history["accuracy"],label="train acc")
plt.plot(gender.history["val_accuracy"],label="val acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.show()


In [ ]:
## gender results
results= model_gender.evaluate(x_test,y_test)
print("Results: ",results)

In [ ]:
## Train-Test Split for age
x_train,x_test,y_train,y_test=train_test_split(x_data,age,test_size=0.2,
                                               shuffle=True,random_state=42)

#one hot encoding
y_train= to_categorical(y_train,num_classes=17) 
y_test= to_categorical(y_test,num_classes=17) 

print("Samples in Training:",x_train.shape)
print("Samples in Testing:",x_test.shape)


print("y_train shape: ",y_train.shape)
print("y_test shape: ",y_test.shape)

input_shape= x_train.shape[1:] 
print("input shape: ",input_shape)

In [ ]:
vgg_layer_list= vgg.layers

model_age= Sequential()

for layer in vgg_layer_list: 
    model_age.add(layer)       

for layer in model_age.layers:
    layer.trainable= False  

model_age.add(layers.Flatten())
model_age.add(layers.Dense(512, activation="relu"))
model_age.add(layers.Dropout(0.5))
model_age.add(layers.Dense(256, activation="relu"))
model_age.add(layers.Dense(128, activation="relu"))
model_age.add(layers.Dense(64, activation="relu"))
model_age.add(layers.Dense(32, activation="relu"))
model_age.add(layers.Dense(17, activation="softmax"))

print(model_age.summary())

In [ ]:
#run model for age
model_age.compile(loss="categorical_crossentropy",
              optimizer= "adam",
              metrics=["accuracy"])

age= model_age.fit(x_train,y_train,
                validation_data=(x_test,y_test),
                epochs=10,
                batch_size= 64)

In [ ]:
#Train and Validation Loss
plt.plot(age.history["loss"],label="train loss")
plt.plot(age.history["val_loss"],label="val loss")
plt.legend()
plt.show()


##Train and Validation Accuracy
plt.plot(age.history["accuracy"],label="train acc")
plt.plot(age.history["val_accuracy"],label="val acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
## age results
results= model_age.evaluate(x_test,y_test)
print("Results: ",results)